In [13]:
import pandas as pd
import re

df = pd.read_csv("tiktok_comments.csv")
df = df.dropna(subset=["text"])
df["text"] = df["text"].astype(str)


In [14]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\d+", "", text)
    return text.strip()

df["clean_text"] = df["text"].apply(clean_text)


In [15]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

embeddings = model.encode(
    df["clean_text"].tolist(),
    show_progress_bar=True
)


Batches: 100%|██████████| 1275/1275 [03:51<00:00,  5.50it/s]


In [16]:
import hdbscan

clusterer = hdbscan.HDBSCAN(
    min_cluster_size=15,
    metric="euclidean",
    cluster_selection_method="eom"
)

clusters = clusterer.fit_predict(embeddings_umap)

df["cluster"] = clusters
print(df["cluster"].value_counts())


cluster
-1      16466
 6       2003
 28       627
 358      611
 64       550
        ...  
 325       16
 51        16
 388       15
 375       15
 270       15
Name: count, Length: 394, dtype: int64


In [17]:
import umap

umap_model = umap.UMAP(
    n_neighbors=15,
    n_components=5,
    metric="cosine",
    random_state=42
)

embeddings_umap = umap_model.fit_transform(embeddings)


c:\Users\User\miniconda3\envs\tiktok_nlp\lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [18]:
import hdbscan

clusterer = hdbscan.HDBSCAN(
    min_cluster_size=15,
    min_samples=5,
    metric="euclidean",
    cluster_selection_method="eom"
)

df["cluster"] = clusterer.fit_predict(embeddings_umap)

print(df["cluster"].value_counts().head(10))


cluster
-1      14352
 7       2003
 457     1192
 28       627
 79       568
 124      476
 441      390
 430      381
 155      304
 166      295
Name: count, dtype: int64


In [19]:
def show_cluster(c, n=5):
    print(f"\nCLUSTER {c}")
    samples = df[df["cluster"] == c]["text"].head(n)
    for s in samples:
        print("-", s)

# Xem 10 cluster lớn nhất
for c in df["cluster"].value_counts().head(10).index:
    show_cluster(c)



CLUSTER -1
- quốc kang mua bùa đâu mà mạnh v
- lại dính bùa
- Bùa mạnh vậy
- quen hở
- Bùa có hạn ạ😞

CLUSTER 7
- =))))
- 😫
- 😂
- 😂😂😂
- 🥰

CLUSTER 457
- 4ph-))
- lô chị
- Lạy bà luôn á :))
- Dạ Lâm đâu r?💁🏼
- chịu chj mun r lại dính bùa

CLUSTER 28
- đang buồn mà xem phải bật cười
- t đùa
- Một lần người ta, một lần chúng ta . Không nên cười nói chuyện của người ta .minh cũng đang khổ sao lũ đố .hãy nên làm việt thiện ♥️
- Cười xỉu
- Hài phết

CLUSTER 79
- Nữa hả🙂 [sticker]
- quoccccc kangggg🥰 [sticker]
- quốc kang🤯🤯 [sticker]
- bùa mới hồi chiêu à [sticker]
- 2vk này:)) [sticker]

CLUSTER 124
- ai bảo lấy chồng muộn => là người lấy chồng sớm nhất nhóm bạn😇
- 😂 Chỉ bảo lấy chồng muộn nhưng
- 😞 đam mê lấy chồng tính đẻ con 25 tuổi và giờ sắp 27 còn chưa lấy ai
- 20/10 còn bảo nào 2 người cưới, kêu còn lâu mới cưới. 2 tuần sau mời cưới t luôn, đáng iu. T ăn đẫm cỗ 27,28
- còn đứa nào nói ko lấy ck là bs bảo cưới 🤣🤣🤣☺️

CLUSTER 441
- chịuu em rồi
- 64 ký vừa không ạ
- mua ntn.
- Xin giá 

In [10]:
cluster_sizes = df["cluster"].value_counts().sort_values(ascending=False)
print(cluster_sizes.head(15))

cluster
-1      16466
 6       2003
 28       627
 358      611
 64       550
 390      496
 108      476
 379      370
 107      307
 130      277
 175      276
 391      266
 208      264
 323      258
 128      244
Name: count, dtype: int64


In [11]:
def show_cluster(c, n=5):
    samples = df[df["cluster"] == c]["text"].head(n)
    print(f"\nCLUSTER {c}")
    for s in samples:
        print("-", s)

# Xem 10 cluster lớn nhất
for c in cluster_sizes.head(10).index:
    show_cluster(c)


CLUSTER -1
- quốc kang mua bùa đâu mà mạnh v
- lại dính bùa
- Bùa mạnh vậy
- quen hở
- Bùa có hạn ạ😞

CLUSTER 6
- =))))
- 😫
- 😂
- 😂😂😂
- 🥰

CLUSTER 28
- đang buồn mà xem phải bật cười
- t đùa
- Một lần người ta, một lần chúng ta . Không nên cười nói chuyện của người ta .minh cũng đang khổ sao lũ đố .hãy nên làm việt thiện ♥️
- Cười xỉu
- Hài phết

CLUSTER 358
- có đý đâu😂
- @Chi không béo @Thu Nguyệt tao đó
- KHÔNG TRÌNH
- không thích mỹ và trung cho lắm , nhưng theo mình mỹ hơn Trung quốc 1 bậc
- Thằng em giỏi nhiwng hình như em chưa nhận xét đủ đâu!

CLUSTER 64
- Nữa hả🙂 [sticker]
- quoccccc kangggg🥰 [sticker]
- quốc kang🤯🤯 [sticker]
- bùa mới hồi chiêu à [sticker]
- 2vk này:)) [sticker]

CLUSTER 390
- Lạy bà luôn á :))
- Dạ Lâm đâu r?💁🏼
- chịu chj mun r lại dính bùa
- Xin giáa
- Xin gia

CLUSTER 108
- ai bảo lấy chồng muộn => là người lấy chồng sớm nhất nhóm bạn😇
- 😂 Chỉ bảo lấy chồng muộn nhưng
- 😞 đam mê lấy chồng tính đẻ con 25 tuổi và giờ sắp 27 còn chưa lấy ai
- 20/10 còn bảo n

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=3000)

for c in cluster_sizes.head(10).index:
    texts = df[df["cluster"] == c]["clean_text"]
    if len(texts) < 5:
        continue
    X = vectorizer.fit_transform(texts)
    keywords = vectorizer.get_feature_names_out()[:10]
    print(f"Cluster {c} keywords:", keywords)

Cluster -1 keywords: ['aa' 'abang' 'ac' 'acc' 'ad' 'ada' 'add' 'adidaphat' 'admin' 'ae']
Cluster 6 keywords: ['uk']
Cluster 28 keywords: ['abit' 'ad' 'ae' 'af' 'ah' 'ai' 'aint' 'allah' 'always' 'am']
Cluster 358 keywords: ['aaa' 'absolutamente' 'ad' 'add' 'adidaphat' 'aem' 'afrique' 'ah' 'ahh'
 'ai']
Cluster 64 keywords: ['aaaaaaaaaaaaaaaaaaaaa' 'abajo' 'about' 'aduh' 'ae' 'ai' 'aihoi' 'alloh'
 'alo' 'also']
Cluster 390 keywords: ['_viethoang_' 'aa' 'abu' 'abysjvwej' 'acc' 'ad' 'add' 'ae' 'ah' 'ai']
Cluster 108 keywords: ['abang' 'abg' 'ac' 'ae' 'ah' 'ai' 'ak' 'aku' 'an' 'anh']
Cluster 379 keywords: ['aaa' 'aburre' 'acc' 'ad' 'add' 'ae' 'ah' 'ahora' 'ai' 'akala']
Cluster 107 keywords: ['acj' 'ae' 'ah' 'ahahahah' 'ai' 'ak' 'an' 'anbruna' 'anh' 'are']
Cluster 130 keywords: ['action' 'afraid' 'ai' 'ak' 'alarm' 'alguém' 'amon' 'an' 'anh' 'anyone']


In [20]:
cluster_to_behavior = {
    -1: "Spam / Noise",
    0: "Tích cực",
    1: "Đùa vui / Cà khịa",
    2: "Tiêu cực / Toxic",
    3: "Hỏi thông tin",
    4: "Nghi ngờ / Bàn tán",
    # các cluster khác map dần...
}

df["behavior_label"] = df["cluster"].map(cluster_to_behavior)
df["behavior_label"] = df["behavior_label"].fillna("Khác / Ít dữ liệu")


In [21]:
from sklearn.metrics import silhouette_score

mask = df["cluster"] != -1
score = silhouette_score(
    embeddings_umap[mask],
    df.loc[mask, "cluster"]
)

print("Silhouette Score:", score)


Silhouette Score: 0.6204671263694763


In [ ]:
df.to_csv("tiktok_comment_clustered.csv", index=False)